- A function to open home page

In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
import random
import time
import bs4
import pandas as pd
import sys
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
def open_website(URL='https://www.yelp.com/'):
    driver = webdriver.Chrome(executable_path='./chromedriver')
    driver.get(URL)
    return driver

- A function to type hoboken(zipcode 07030) + restaurant

In [3]:
def select_location_business(driver, location_input = '07030', business_type = 'Restaurant'):
    #Option 1 to delay
    normal_delay = random.normalvariate(2, 0.5)
    time.sleep(normal_delay)
    
    #find the location bar
    #you could use another format @ http://selenium-python.readthedocs.io/locating-elements.html
    active_location_search_input = driver.find_element_by_id("dropperText_Mast")
    #type hoboken in the location bar
    active_location_search_input.clear()
    active_location_search_input.send_keys(location_input)
    #find the business bar
    
    
    #Option 2 to delay
    wait = WebDriverWait(driver, 10) #step 1
    #find the location bar
    #other format @ http://selenium-python.readthedocs.io/waits.html
    active_business_search_input = wait.until(EC.element_to_be_clickable((By.ID, "find_desc"))) #step 2
    
    #active_restaurant_search_input = active_business_search_div.find_element_by_xpath()
    #type restaurant in the bar
    active_business_search_input.send_keys(business_type)
    #find the search button
    hit_search = driver.find_element_by_id("header-search-submit")
    #click the search button
    search_result = hit_search.click()
    return driver

### Extract data

we require:
1. restaurant name(Grand Vin)
2. restaurant id 
3. restaurant general star range (4.0)
4. restaurant price range ($$)
5. restaurant category list (Wine Bars, Italian, Cocktail)
6. each review under a restaurant  
    - user name (Jason L.)
    - user id ("/user_details?userid=K58UsGqR6k5lhPZKCVcuRg")
    - review text (...)
    - rating (5.0 star rating)  


- collect_res_id

In [4]:
def extract_id_df(driver, ad_no):
    restaurant_id_xpath_li = [] 
    restaurant_id_li = []
    restaurant_name_li = []
    for i in range(10):
        no = str(i+1+ad_no)
        id_xpath = """//*[@id="super-container"]/div/div[2]/div[1]/div/div[5]/ul[2]/li[{}]/div/div[1]/div[1]/div/div[2]/h3/span"""
        id_xpath = id_xpath.format(no)
        restaurant_id_xpath_li.append(id_xpath)
    for i in range(len(restaurant_id_xpath_li)):
        restaurant_id_element = driver.find_element_by_xpath(restaurant_id_xpath_li[i])
        data_html = restaurant_id_element.get_attribute('innerHTML')
        soup = bs4.BeautifulSoup(data_html,'html5lib')
        restaurant_id_tag = soup.find('a').attrs
        restaurant_id = restaurant_id_tag['data-hovercard-id']
        restaurant_id_li.append(restaurant_id)

        restaurant_name = soup.find('a').text
        restaurant_name_li.append(restaurant_name)
    df = pd.DataFrame(data = {'restaurant_id' : restaurant_id_li, 'restaurant_name':restaurant_name_li})
    return df

- collect res profile

In [5]:
def extract_restaurant_li(driver):
    #res_name
    restaurant_name_element = driver.find_element_by_xpath("""//*[@id="wrap"]/div[2]/div/div[1]/div/div[3]/div[1]/div[1]/h1""")
    restaurant_name = restaurant_name_element.text
    
    #res_rating
    data_element = driver.find_element_by_xpath("""//*[@id="wrap"]/div[2]/div/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]""")
    data_html = data_element.get_attribute('innerHTML')
    soup = bs4.BeautifulSoup(data_html,'html5lib')
    restaurant_rating_tag = soup.find('div').attrs
    restaurant_rating = restaurant_rating_tag['title']
    
    #res_price
    restaurant_price_element = driver.find_element_by_xpath("""//*[@id="wrap"]/div[2]/div/div[1]/div/div[3]/div[1]/div[2]/div[2]/span[1]/span""")
    restaurant_price = restaurant_price_element.text
    
    #res_tag
    restaurant_tag_element = driver.find_element_by_xpath("""//*[@id="wrap"]/div[2]/div/div[1]/div/div[3]/div[1]/div[2]/div[2]/span[2]""")
    restaurant_tag = restaurant_tag_element.text
    
    li = [restaurant_name, restaurant_rating,restaurant_price,restaurant_tag]
    return li

- collect user reviews

In [6]:
def extract_reviews_df(driver):
    name_li = []
    rating_li = []
    text_li = []
    user_id_li = []

    reviews_table_element = driver.find_element_by_class_name("review-list")
    data_html = reviews_table_element.get_attribute('innerHTML')
    soup = bs4.BeautifulSoup(data_html,'html5lib')
    reviews_table = soup.find('ul')
    
    #user_name
    name_tag = reviews_table.find_all('a', attrs = {'class':'user-display-name js-analytics-click'})
    for i in range(len(name_tag)):
        name = name_tag[i].text
        name_li.append(name)
    
    #user_id
    for i in range(len(name_tag)):
        user_id = name_tag[i].attrs['data-hovercard-id']
        user_id_li.append(user_id)

    #user_rating
    reviews_tag = reviews_table.find_all('div',attrs= {"class":'review-content'})
    for i in range(len(reviews_tag)):
        review_rating_tag = reviews_tag[i].find('div',attrs={'class':'i-stars'}).attrs
        review_rating = review_rating_tag['title']
        rating_li.append(review_rating)
    #user_text
    for i in range(len(reviews_tag)):
        review_text_tag = reviews_tag[i].find('p')
        review_text = review_text_tag.text
        text_li.append(review_text)
    
    df = pd.DataFrame(data = {'user_name': name_li, 'user_id':user_id_li,'user_rating':rating_li,'user_text':text_li})
    return df

### Select restaurant
- select each re at one page

In [7]:
def detect_ad_no(driver):
    try:
        data_element = driver.find_element_by_xpath("""//*[@id="super-container"]/div/div[2]/div[1]/div/div[5]/ul[2]""")
        data_html = data_element.get_attribute('innerHTML')
        soup = bs4.BeautifulSoup(data_html,'html5lib')
        ad_list = soup.find_all('li', attrs={'class': 'js-yloca js-yloca-search yloca-search-result'})
        ad_no = len(ad_list)
    except:
        ad_no = 0
    return ad_no

- in_scrape_out

In [8]:
def select_back_all_re(driver):
    restaurant_xpath_li = []
    res_profile_li = []
    ad_no = detect_ad_no(driver)
    id_df = extract_id_df(driver, ad_no)
    try:
        for i in range(3):    
            for i in range(10):
                no = str(i+1+ad_no)
                re_xpath = """//*[@id="super-container"]/div/div[2]/div[1]/div/div[5]/ul[2]/li[{}]/div/div[1]/div[1]/div/div[2]/h3/span/a"""
                re_xpath = re_xpath.format(no)
                restaurant_xpath_li.append(re_xpath)

            for i in range(len(restaurant_xpath_li)):
                normal_delay = random.normalvariate(3, 0.5)
                time.sleep(normal_delay)
                select_business = driver.find_element_by_xpath(restaurant_xpath_li[i])
                click_business = select_business.click()
                res_li = extract_restaurant_li(driver) 
                count = 1
                #next_page
                reviews_df = extract_reviews_df(driver)

                for i in range(50):
                    try:
                        next_button = driver.find_element_by_link_text("""Next""")
                        next_button.click()
                        reviews_df_more = extract_reviews_df(driver)
                        reviews_df = pd.concat([reviews_df, reviews_df_more], axis=0, names=None, ignore_index = True)
                        normal_delay = random.normalvariate(5, 0.5)
                        time.sleep(normal_delay)
                        count += 1
                    except:
                        pass
                res_li = extract_restaurant_li(driver) 
                reviews_df['restaurant_name'] = res_li[0]
                reviews_df['restaurant_rating'] = res_li[1]
                reviews_df['restaurant_price'] = res_li[2]
                reviews_df['restaurant_type'] = res_li[3]
                file_name = str(res_li[0])+('.csv')
                df = reviews_df
                df.to_csv(file_name)

                reviews_df = None
                back_page_no = "window.history.go({})".format(str(-count))
                driver.execute_script(back_page_no)
            next_button = driver.find_element_by_link_text("""Next""")
            next_button.click()
    except:
        pass
    return driver

In [9]:
driver = open_website('https://www.yelp.com/')

In [10]:
driver = select_location_business(driver, '07030', 'Restaurant')

In [11]:
driver = select_back_all_re(driver)

WebDriverException: Message: unknown error: Element <a class="biz-name js-analytics-click" data-analytics-label="biz-name" href="/biz/gogi-grill-hoboken-hoboken-2?osq=Restaurant" data-hovercard-id="6wg-plyXFXT8ozNtG7dXWw">...</a> is not clickable at point (305, 575). Other element would receive the click: <p class="snippet">...</p>
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544337 (8c0344a12e552148c185f7d5117db1f28d6c9e85),platform=Mac OS X 10.13.3 x86_64)


### Main scraper

In [ ]:
def main():
    driver = open_website('https://www.yelp.com/')
    driver = select_location_business(driver, '07030', 'Restaurant')
    driver = select_back_all_re(driver)
    return None

### Run scraper

In [ ]:
if __name__ == '__main__':
    main()